# Proyecto Análisis Aplicado
Jacqueline Lira Chávez **167334**  

El objetivo de este proyecto es implementar el algoritmo de Búsqueda Lineal de Newton con Modificación a la Hessiana

## Código

#### Imports

In [1]:
import numpy as np
from random import random
from IPython.display import display, Math

#### Gradiente
Código que calcula el gradiente de la función f en el punto xk utilizando la derivada mediante la diferencia central.
$$f'(x_k) = \frac{f(x_k+k)-f(x_k-k)}{2k}$$

In [2]:
# Calcu
def gradiente(f, xk, eps = 1e-6):
    size = len(xk)
    grad = np.zeros(size)
    for i in range(size):
        x_f = xk.copy(); x_f[i] += eps
        x_b = xk.copy(); x_b[i] -= eps
        grad[i] = (f(x_f) - f(x_b)) / (2 * eps)
    return grad

#### Hessiana
Código que calcula el hessiana de la función f en el punto $x_k$ utilizando la derivada mediante la diferencia 
central, utilizando la función gradiente. $$f''(x_k) = \frac{f'(x_k+h)-f'(x_k-h)}{2h}$$

In [3]:
def hessiana(f,xk, eps = 1e-3):
    size = len(xk)
    hess = np.zeros((size, size))
    for j in range(size):
        # Primera derivada
        dx_ff = xk.copy(); dx_ff[j] += eps;
        dx_bb = xk.copy(); dx_ff[j] += eps;
        grad_f = gradiente(f, dx_ff)
        grad_b = gradiente(f, dx_bb)
        for i in range(j+1):
            # Segunda derivada
            hess[i, j] = (grad_f[i] - grad_b[i]) / (2 * eps)
            hess[j, i] = (grad_f[i] - grad_b[i]) / (2 * eps)
    return hess

#### Backtracking Line Search (Algoritmo 3.1)
Algoritmo que determina la cantidad a moverse dentro de una dirección de búsqueda.
Notamos que para métodos quasi Newton se útiliza el valor inicial $\alpha_0 = 1$

In [4]:
def BLS(f, xk, alpha_0, pk, c):
    alpha = alpha_0
    while f(xk + alpha * pk) > f(xk) + c * alpha * gradiente(f, xk) @ pk:
        p = random()
        alpha = p * alpha
    return alpha

#### Cholesky with Added Múltiple of the Identity (Algoritmo 3.3)
Buscamos $\tau$ de tal forma que la matriz $A + \tau I$ es positiva semidefinida, por lo que iterativamente se va aumentando $\tau$ hasta que se cumpla

In [5]:
def CAMI(A, beta = 1e-4, max_iter = 10):
    n = len(A)
    if min(np.diag(A)) > 0:
        t = 0
    else:
        t = -min(np.diag(A)) + beta
    for k in range(max_iter):
        try:
            X = np.linalg.cholesky(A + t * np.identity(n))
        except np.linalg.LinAlgError as err:
            t = max(2 * t, beta)
        else:
            break
    return A + t * np.identity(n)

#### Line Search Newton with Modification (Algoritmo 3.2)
Se le hace una modificación al algoritmo de Newton para que se puede utilizar aunque la matriz hessiana no sea semipositiva definida.

In [6]:
def LSNM(f, xk):
    for k in range(100):
        Bk = hessiana(f, xk)
        # Calculamos si es semipositiva definida
        try:
            np.linalg.cholesky(Bk)
        except np.linalg.LinAlgError as err:
            # Si no lo es, la convertimos en matriz semipositiva
            Bk = CAMI(Bk)   
        # Aplicamos el método de Newton
        pk = np.linalg.solve(Bk, - gradiente(f, xk))
        xk = xk + BLS(f, xk, 1, pk, 0.5) * pk
    return xk

## Pruebas
Para probar nuestro algoritmo utilizamos la función de Rosenbrock definida como $f(x,y)=(a-x)^2+b(y-x^2)^2$, que tiene un mínimo local en $(a,a^2)$, con diversos valores para $a$ y $b$ y puntos iniciales $(x,y)$

#### Función de Rosenbrock

In [7]:
def Rosenbrock(x, a = 1, b = 100):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

##### Prueba 1
$(a,b) = (1, 100)$

$x = (0.9, 1.1)$

Primero probamos con un valor cercano al verdadero.

In [8]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

x = np.array([0.9, 1.1])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba 2
$(a,b) = (1, 100)$

$(x,y) = (8, -4)$  

Utilizamos un vector más alejado del óptimo, en dónde también podemos ver que funcione correctamente el algoritmo de Cholesky con Identidad Múltiple Agregada

In [9]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba 3
$(a,b) = (2, 100)$

$(x,y) = (8, -4)$  

Utilizamos un vector más alejado del óptimo, en dónde también podemos ver que funcione correctamente el algoritmo de Cholesky con Identidad Múltiple Agregada

In [10]:
def Rosenbrock_2(x, a = 2, b = 50):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_2(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock_2, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_2(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba 4
$(a,b) = (-2, 5)$

$(x,y) = (8, -4)$  

Utilizamos un vector más alejado del óptimo, en dónde también podemos ver que funcione correctamente el algoritmo de Cholesky con Identidad Múltiple Agregada

In [11]:
def Rosenbrock_3(x, a = -2, b = 5):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_3(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock_3, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_3(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>